## New Features

source: kaggle discussion
- late payment
- credit utilization
- debt ratio

In [42]:
#pacakges import
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

In [43]:
install = pd.read_csv('data/installments_payments.csv')
buro = pd.read_csv('data/bureau.csv')
buro_bal = pd.read_csv('data/bureau_balance.csv')

## 1. Late Payment
---
installment_payments.csv
- Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
- There is a) one row for every payment that was made plus b) one row each for missed payment.
- One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.  

**Columns used here:**  
- DAYS_INSTALMENT: When the installment of previous credit was supposed to be paid (relative to application date of current loan)  
- DAYS_ENTRY_PAYMENT: When was the installments of previous credit paid actually (relative to application date of current loan)

**New feature: LATE_PAYMENT**  
installment['DAYS_INSTALMENT']-installment['DAYS_ENTRY_PAYMENT'] means the number of days delayed  

In [44]:
# installment_temp = installment[installment.DAYS_ENTRY_PAYMENT >= -365]
install['LATE_PAYMENT'] = install['DAYS_INSTALMENT']-install['DAYS_ENTRY_PAYMENT']

In [45]:
install.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,LATE_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,7.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,8.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-17.0


In [46]:
late_payment_feature = install.groupby('SK_ID_CURR')[['LATE_PAYMENT']].min().reset_index()
late_payment_feature.head()

,SK_ID_CURR,LATE_PAYMENT
0,100001,-11.0
1,100002,12.0
2,100003,1.0
3,100004,3.0
4,100005,-1.0


In [47]:
late_payment_feature.isnull().sum()

SK_ID_CURR      0
LATE_PAYMENT    9
dtype: int64

## 2. Credit Utilization
---
According to credit scoring papers, credit utilization is a strong indicator for a risky customer  
credit utilization can be calculated by dividing Credit card balance by credit card limit  
About the data:
credit_card_balance.csv

- Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample   
- i.e. the table has
* #of loans in sample 
* #of relative previous credit cards 
* #of months where we have some history observable for the previous credit card) rows.


AMT_BALANCE: Balance during the month of previous credit  
AMT_CREDIT_LIMIT_ACTUAL: Credit card limit during the month of the previous credit  


In [48]:
cred = pd.read_csv('data/credit_card_balance.csv')
cred.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [49]:
month = -2 
cred_temp = cred[cred.MONTHS_BALANCE >= month]
cred_temp['CRED_UTIL'] = cred_temp['AMT_BALANCE'] / (1+cred_temp['AMT_CREDIT_LIMIT_ACTUAL'])
cred_util_feature = cred_temp.groupby('SK_ID_CURR')['CRED_UTIL'].max().reset_index().rename(columns={'CRED_UTIL':'CRED_UTIL_'+str(month*-1)})
cred_util_feature.head()

,SK_ID_CURR,CRED_UTIL_2
0,100006,0.000000
1,100011,0.000000
2,100013,0.000000
3,100021,0.000000
4,100028,0.165937


## 3. Debt Ratio

In [50]:
buro['DEBT_RATIO']=buro['AMT_CREDIT_SUM_DEBT']/(1+buro['AMT_CREDIT_SUM'])
debt_ratio_feature = buro.groupby('SK_ID_CURR')['DEBT_RATIO'].max().reset_index()
debt_ratio_feature.head()

,SK_ID_CURR,DEBT_RATIO
0,100001,0.987402
1,100002,0.546179
2,100003,0.000000
3,100004,0.000000
4,100005,0.954793


## Merge with application_train

In [51]:
train = pd.read_csv('data/application_train.csv')

In [52]:
original_features = list(train.columns)
print('Original Number of Features: ', len(original_features))

# Merge with the value counts of bureau
train = train.merge(late_payment_feature, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
train = train.merge(cred_util_feature, on = 'SK_ID_CURR', how = 'left')

# Merge with the monthly information grouped by client
train = train.merge(debt_ratio_feature, on = 'SK_ID_CURR', how = 'left')

print('Number of Features Now: ', len(train.columns))

Original Number of Features:  122
Number of Features Now:  125


In [53]:
domain_features = train.drop(columns = original_features)
domain_features["SK_ID_CURR"] = train["SK_ID_CURR"]
domain_features.head()

,LATE_PAYMENT,CRED_UTIL_2,DEBT_RATIO,SK_ID_CURR
0,12.0,NaN,0.546179,100002
1,1.0,NaN,0.000000,100003
2,3.0,NaN,0.000000,100004
3,1.0,0.0,NaN,100006
4,-12.0,NaN,0.000000,100007


## Deal with missing values

In [54]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
missing_values_table(domain_features)

Your selected dataframe has 4 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
CRED_UTIL_2,227419,74.0
DEBT_RATIO,51381,16.7
LATE_PAYMENT,15876,5.2


In [55]:
missing_col = domain_features["CRED_UTIL_2"].isnull()
domain_features.loc[missing_col,"CRED_UTIL_2_missing"] = 1
domain_features["CRED_UTIL_2_missing"].fillna(0,inplace=True)
domain_features.head()

,LATE_PAYMENT,CRED_UTIL_2,DEBT_RATIO,SK_ID_CURR,CRED_UTIL_2_missing
0,12.0,NaN,0.546179,100002,1.0
1,1.0,NaN,0.000000,100003,1.0
2,3.0,NaN,0.000000,100004,1.0
3,1.0,0.0,NaN,100006,0.0
4,-12.0,NaN,0.000000,100007,1.0


In [56]:
domain_features.fillna(domain_features.median(),inplace=True)
domain_features.head()

,LATE_PAYMENT,CRED_UTIL_2,DEBT_RATIO,SK_ID_CURR,CRED_UTIL_2_missing
0,12.0,0.0,0.546179,100002,1.0
1,1.0,0.0,0.000000,100003,1.0
2,3.0,0.0,0.000000,100004,1.0
3,1.0,0.0,0.760045,100006,0.0
4,-12.0,0.0,0.000000,100007,1.0


In [57]:
missing_values_table(domain_features)

Your selected dataframe has 5 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [58]:
domain_features.to_csv("domain_features.csv",index=False)

In [59]:
domain_features

,LATE_PAYMENT,CRED_UTIL_2,DEBT_RATIO,SK_ID_CURR,CRED_UTIL_2_missing
0,12.0,0.000000,0.546179,100002,1.0
1,1.0,0.000000,0.000000,100003,1.0
2,3.0,0.000000,0.000000,100004,1.0
3,1.0,0.000000,0.760045,100006,0.0
4,-12.0,0.000000,0.000000,100007,1.0
5,-1317.0,0.000000,0.897051,100008,1.0
6,-7.0,0.000000,0.967784,100009,1.0
7,5.0,0.000000,0.515566,100010,1.0
8,-958.0,0.000000,0.000000,100011,0.0
9,-14.0,0.000000,0.760045,100012,1.0
